In [ ]:
!pip install requests
!pip install pandas
!pip install bs4

In [49]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import configparser
import json

In [7]:
parser = configparser.ConfigParser()
parser.read("./secrets.ini")
foia_api = parser['foia']["api"]

In [42]:
# https://www.foia.gov/swagger.html, https://www.foia.gov/developer/agency-api/
total_pages = 20
documents = {'objects': []}
for page in range(total_pages):
    r = requests.get(f"https://www.cia.gov/readingroom/search/site?page={page}&f%5B0%5D=im_field_taxonomy_keyword%3A477")
    soup = BeautifulSoup(r.text, 'html.parser')
    try:
        lines = soup.find('ol', {'class':"search-results apachesolr_search-results"}).findAll('li')
        for line in lines:
            html_link = line.find('a')
            link = html_link['href']
            text = html_link.text
            documents['objects'].append({"name": text, "link": link})
    except:
        pass

## Organize Requests

CIA FOIA Most recent UFO Requests

In [47]:
for i, object in enumerate(documents['objects']):
    link = object['link']
    r_1 = requests.get(link)
    soup_1 = BeautifulSoup(r_1.text, 'html.parser')
    main = soup_1.find('span', {'class':'file'})
    pdf_document = main.find('a')['href']
    documents['objects'][i]['pdf_document'] = pdf_document

In [50]:
with open("cia_foia_ufo.json", "w") as f:
    json.dump(documents, f)